In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
GROQ_API_KEY = os.environ["GROQ_API"]

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from pydantic import BaseModel



# Pdf Loader

In [75]:
# loader = PyPDFLoader(r"LLM_based_web_agents.pdf")
# loaded_document = loader.load()

# Load multiple Docx Files


In [5]:
from langchain_community.document_loaders import Docx2txtLoader, DirectoryLoader


In [7]:
#Extract Data From the PDF File
def load_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [8]:
loaded_data=load_file(data='data/')

In [9]:
len(loaded_data)

637

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [19]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=100)
chunks_of_text = text_splitter.split_documents(loaded_data)

In [20]:
len(chunks_of_text)

2785

In [14]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [15]:
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\PMLS\anaconda3\envs\llmapp\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [21]:

vector_db = FAISS.from_documents(chunks_of_text, embed_model)

In [22]:
vector_db

In [23]:
retriever = vector_db.as_retriever()

In [24]:
response = retriever.invoke("Tell me about hypoglycemia?")

In [25]:
response

[Document(metadata={'source': 'data\\Medical_book.pdf', 'page': 275}, page_content='Sulonylureas. All sulfonylurea drugs may cause\nhypoglycemia. Most patients become resistant to thesedrugs over time, and may require either dose adjustmentsor a switch to insulin. The list of adverse reactions isextensive, and includes central nervous system problemsand skin reactions, among others. Hematologic reactions,although rare, may be severe and include aplastic ane-\nmia and hemolytic anemia . The administration of oral\nhypoglycemic drugs has been associated with increasedcardiovascular mortality as compared with treatmentwith diet alone or diet plus insulin. The sulfonylureas areclassified as category C during pregnancy, based on ani-\nGALE ENCYCLOPEDIA OF MEDICINE 2 262Antidiabetic drugsGEM - 0001 to 0432 - A  10/22/03 1:42 PM  Page 262'),
 Document(metadata={'source': 'data\\Medical_book.pdf', 'page': 146}, page_content='Altitude sickness occurs because the partial pressure\nof oxygen decr

In [26]:
len(response)

4

In [27]:
retriever = vector_db.as_retriever(search_kwargs={"k": 1})

In [28]:
response = retriever.invoke(" what are the symptoms of hypoglycemia?")
response

[Document(metadata={'source': 'data\\Medical_book.pdf', 'page': 195}, page_content='• a ruptured blood vessel\n• surgery\nWhen a lot of blood is lost within a short time, blood\npressure and the amount of oxygen in the body drop sud-denly. Heart failure and death can follow.\nLoss of even one-third of the body’s blood volume in\nthe space of several hours can be fatal. More gradual bloodloss is less serious, because the body has time to createnew red blood cells to replace those that have been lost.\nSymptoms\nWeakness, fatigue, and a run-down feeling may be\nsigns of mild anemia. Skin that is pasty or sallow, or lackof color in the creases of the palm, gums, nail beds, orlining of the eyelids are other signs of anemia. Someonewho is weak, tires easily, is often out of breath, and feelsfaint or dizzy may be severely anemic.\nOther symptoms of anemia are:\n• angina pectoris (chest pain, often accompanied by a\nchoking sensation that provokes severe anxiety )\n• cravings for ice, paint, 

## Simple use with LCEL

In [29]:
#from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [30]:
template = """Answer the question based only on the following context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# model = ChatOpenAI()




In [31]:
from langchain_groq import ChatGroq



In [32]:
from groq import Groq

In [33]:
llm = ChatGroq(
    api_key = GROQ_API_KEY,
    model="llama-3.1-8b-instant",
    disable_streaming=True
)

# LCEL Chaining

In [34]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [29]:
def chatbot_Func():
 while True:
  question = input("You: ")
  if question.lower() == "exit":
    break 
  response = chain.invoke(question)
  print(f"AI: {response}")

In [ ]:
chatbot_Func()

In [35]:
# query = "who is the founder of css and in which year it was founded?"
query = input("Enter your query: ")


retriever = vector_db.as_retriever(search_kwargs={"k": 3})

docs = retriever.invoke(query)

In [36]:
# Function to format documents into a string
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

formatted_response = format_docs(docs)
# print(formatted_response)

In [37]:
from langchain.schema import HumanMessage

In [38]:
# Prepare the input as a list of messages for the chat model
messages = [
    HumanMessage(content=f"Question:{query}\n\ncontext:{formatted_response}") 
    ]



In [39]:
chunks = []
for chunk in llm.stream(messages):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)

The provided text does not discuss hypoglycemia, but rather anemia, anoxia, and hypoxia. However, considering the context of a ruptured blood vessel and surgery, which can lead to blood loss and shock, hypoglycemia is a possible complication.

Hypoglycemia is a condition characterized by low blood sugar levels (less than 70 mg/dL). Symptoms of hypoglycemia may include:

1. Shakiness or tremors
2. Sweating
3. Hunger or nausea
4. Dizziness or lightheadedness
5. Confusion or disorientation
6. Headache
7. Fatigue or weakness
8. Rapid heartbeat
9. Rapid breathing
10. Pale or cool skin
11. Numbness or tingling in the lips, tongue, or fingers
12. Difficulty speaking or slurred speech
13. Loss of consciousness or coma (in severe cases)

If the situation is caused by a ruptured blood vessel or surgery, symptoms may be more severe due to the associated blood loss and shock. In such cases, rapid diagnosis and treatment are critical to prevent further complications and potential organ damage.

# Memory

In [40]:
# Initialize an empty string to store chat history
bot_memory = ""

# Combine chat history (bot_memory) with the new formatted response
context = bot_memory + "\n" + formatted_response

chain = (
    {
        "context": RunnablePassthrough(),  # Ensure it's treated as a passthrough Runnable
        "question": RunnablePassthrough()  # Allow direct question passthrough
    }
    | prompt  # Apply the chat prompt template
    | llm  # Invoke the LLM model
    | StrOutputParser()  # Parse the LLM response as a string
)

In [ ]:
def chatbot():
      # Initialize an empty string to store chat history
    bot_memory = ""


    print("Welcome to the medical website chatbot! Type 'exit' to quit.")

    while True:
        # Get user input
        user_input = input("You: ")

        # Exit condition
        if user_input.lower() == 'exit':
            print("Goodbye! Have a nice day!")
            break
        
        else:
            retriever = vector_db.as_retriever(search_kwargs={"k": 3})

            docs = retriever.invoke(user_input)
            
            formatted_response = format_docs(docs)

        # Combine chat history (bot_memory) with the new formatted response
        context = bot_memory + "\n" + formatted_response
        # Invoke the chain with the user input and context
        FinalResponse = chain.invoke({"question": user_input, "context": context})

        # Display the chatbot's response
        print(f"AI: {FinalResponse}")

        # Update the bot's memory with the conversation
        bot_memory += f"\nUser: {user_input}\nAI: {FinalResponse}\n"
        FinalResponse


In [42]:
chatbot()


Welcome to the CSS chatbot! Type 'exit' to quit.
AI: The reasons of liver pain mentioned in the context are:

1. Blood disorders such as polycythemia rubra vera (an increase in the number of red blood cells) and sickle cell disease.
2. Liver cancer, which is the most frequent cause in parts of the world where liver cancer is common.
3. Certain infections.
4. Use of oral contraceptives.
5. Decreased urine output.
6. Cirrhosis of the liver, which is often caused by chronic alcohol abuse and leads to scarring of the liver and an increase in blood pressure in the portal veins.
7. Portal hypertension, which can occur due to cirrhosis or portal vein obstruction from unknown causes. This leads to blood flow backward, causing the portal veins to enlarge and the emergence of bleeding varices across the esophagus and stomach.
AI: The reasons of liver pain mentioned in the context are:

1. Blood disorders such as polycythemia rubra vera (an increase in the number of red blood cells) and sickle ce